# GMAP package metadata form

| Field | Field description (and example entries) |
| --- | --- |
| Map name (GMAP_ID) | Unique package name (`GMAP-{{target-body}}-{{content-type}}-{{label}}`) |
| Target body | Name of target body (eg, `Mercury`) |
| Title of map | Map title (eg, `Awesome Geologic Map of the region X`) |
| Bounding box - Min Lat | Minimum latitude in degrees [-90:90) (< Max Lat) |
| Bounding box - Max Lat | Maximum latitude in degrees (-90:90] (> Min Lat) |
| Bounding box - Min Lon | West-most Longitude in degrees [-180:180) (< Max Lon) |
| Bounding box - Max Lon | East-most Longitude in degrees (-180:180] (> Min Lon) |
| Author(s) | Semi-colon separated list of authors |
| Type | Either "draft" or "released" |
| Output scale | Map spatial scale |
| Original Coordinate Reference System | WKT declaring map' CRS |
| Data used | Semi-colon separated list of ancillary, original data used |
| Standards adhered to | Semi-colon list of standards used in the map |
| DOI of companion paper(s) | DOI of linked publication |
| Aims | Reason, goal for this map |
| Short description | Free-text (500 words maximum) describing the map |
| Related products | Other geological maps complementing this one |
| Units Definition (polygon styling) | Units color definition |
| Stratigraphic info | Description of stratigraphic elements in the map |
| Other comments | free-text (notes, errata, warnings) |
| Heritage used | heritage information |
| Link to other data | Links to extenal resources |
| Acknowledgements | Free-text acknowledge |

In [1]:
# json_schema = {
#   # "$id": "planmap.schema.json",

#   "title": "PLanmap package schema",
#   "$comment": "Define Planmap data package metadata",

#   "type" : "object",

#   "properties": {
#       "pm_id": { "type" : "string" },
      
#       "title": { "type" : "string" },
      
#       "target": { 
#           "enum": ["mars","mercury","moon","venus"] 
#       },
      
#       "authors": { 
#           "type": "array",
#           "items": {
#               "type": "string"
#           },
#           "minItems": 1
#       },
      
#     "bbox_lat_min": { 
#       "type" : "number",
#       "minimum": -90,
#       "maximum": 90
#      },
#     "bbox_lat_max": { 
#       "type" : "number",
#       "minimum": -90,
#       "maximum": 90
#      },
#     "bbox_lon_west": { 
#       "type" : "number",
#       "minimum": -180,
#       "maximum": 180
#      },
#     "bbox_lon_east": { 
#       "type" : "number",
#       "minimum": -180,
#       "maximum": 180
#      },
      
#       "crs": { "type": "string" },
      
#       "output_scale": { "type": "string" }
#   },

# #   "patternProperties": {
# #     "^Aims": { "type": "string" },
# #     "^Author": { "type" : "string" },
# #     "^Acknowledgements": { "type": "string" },
# #     "^DOI" : { "type": "string" },
# #     "^Link": { "type": "string" },
# #     "^Map name": { "type": "string" },
# #     "^Other comments": { "type": "string" },
# #     "^Related products": { "type": "string" },
# #     "^Standards": { "type": "string" },
# #     "^Stratigraphic info": { "type": "string" },
# #     "^Units Definition": { "type": "string" },

# #   },

# #   "additionalProperties": { "type" : "boolean" }
# }

In [2]:
layout = [
    'title',
    'shortname',
    'map_type',
    'target',
    [
        {"Longitude (west,east) [-180:180]": ['bbox_lon_west', 'bbox_lon_east']},
        {"Latitude (min,max) [-90:90]": ['bbox_lat_min', 'bbox_lat_max']}
    ],
    'publication_date',
    'doi',
    'authors',
    [
        # {"Map description": ['description', 'aims', 'units', 'stratigraphic_info']},
        # {"Spatial attributes": ['crs', 'output_scale']},
        # {"Ancillary data": ['ancillary_data', 'related_products', 'heritage', 'extra_data']},
        # {"Notes": ['standards', 'comments', 'acknowledge']}
    ],
    'gmap_id'
]

schema = {
    "type": "object",
    
    "properties": {
        'title': {
            "type": "string",
            "description": "Map title"
        },

        'authors': {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },

        'bbox_lon_west': {
            "description": "West Longitude",
            "type": "number",
            "minimum": -180,
            "maximum": 180
        }, 

        'bbox_lon_east': {
            "description": "East Longitude",
            "type": "number",
            "minimum": -180,
            "maximum": 180
        },

        'bbox_lat_min': {
            "description": "Min Latitude",
            "type": "number",
            "minimum": -90,
            "maximum": 90
        }, 

        'bbox_lat_max': {
            "description": "Max Latitude",
            "type": "number",
            "minimum": -90,
            "maximum": 90
        }, 

        'publication_date': {
            "description": "Publication date",
            "type": "string",
            "format": "date"
        },

        'doi': {
            "type": "string",
        },

        'crs': {
            "type": "string",
        },

         'shortname': {
            "type": "string"
        },

        'map_type': {
            "description": "Map type",
            "type": "array",
            "items": {
                "type": "string",
                "enum": [
                    "Integrated",
                    "Morphologic",
                    "Compositional",
                    "Digital model",
                    "Stratigraphic",
                    "Geo-structural",
                ]
            },
            "minItems": 1,
            "uniqueItems": True
        },

        'target': {
            "description": "Target body",
            "type": "string",
            "enum": ['Mars', 'Mercury', 'Moon', 'Venus']
        },

       'gmap_id': {
            "type": "string",
            "readOnly": True
        }
    },
    
    "required": [
        "gmap_id",
        "target",
        "map_type",
        "shortname",
        "title",
    ]
}

In [20]:
# from api.gui import display,form
from api.gui import form
from IPython.display import display, JSON

form = form.Form(schema=schema, layout=layout)

def on_change_value(change):
    """
    Set (or reset) 'gmap_id' when 'target','map_type','shortname' change values
    """
    map_types = {
        "Integrated": "I",
        "Morphologic": "M",
        "Compositional": "C",
        "Digital model": "D",
        "Stratigraphic": "S",
        "Geo-structural": "G",
    }

    _target = form['target'].value.strip()
    _type = ''.join([ map_types[t].upper() for t in form['map_type'].value ])
    _sname = form['shortname'].value.strip().replace(' ','-')
    _id = ["GMAP", _type, _target, _sname]
    
    if not all(_id):
        form['gmap_id'].value = ""
    else:
        form['gmap_id'].value = '_'.join(_id)

# Link the widgets to trigger the function above ('on_change_value')
#
form['target'].observe(on_change_value, names='value')
form['shortname'].observe(on_change_value, names='value')
form['map_type'].observe(on_change_value, names='value')

# form.set_link(source=['target','map_type','shortname'], destiny='gmap_id', trigger=on_change_value)

from ipywidgets import widgets

output_panel = widgets.Output(layout={'border': '1px solid black'})

@output_panel.capture(clear_output=True)
def validate_btn(*args,**kargs):
    import json
    # print(json.dumps(form.to_json(), indent=2))
    display(JSON(form.to_json()))
        
btn_to_json = widgets.Button(icon="save")
btn_to_json.on_click(validate_btn)

header = widgets.TwoByTwoLayout(
    bottom_right = btn_to_json
)

app = widgets.AppLayout(
    header = header,
    left_sidebar = form.widget,
    center = None,
    right_sidebar = output_panel,
    footer = None
)

In [21]:
form.read_json({
    'shortname': 'Some name',

    'map_type': [
                "Integrated",
                "Morphologic",
                "Compositional",
                "Digital model",
                "Stratigraphic",
                "Geo-structural",
            ],

    'target': 'Mars',
    
    'authors': ['zezinho'],

    'title': 'A map'
})

In [22]:
display(app)

AppLayout(children=(TwoByTwoLayout(children=(Button(icon='save', layout=Layout(grid_area='bottom-right'), styl…

In [24]:
form.set('doi','bla')

form.set('target','Venus')

form.set('shortname','Some region')

form.set('map_type',['Morphologic','Stratigraphic'])

form.set('authors',['bla','ble'])

In [23]:
form.to_json()

{'title': 'A map',
 'authors': ['zezinho'],
 'bbox_lon_west': 0.0,
 'bbox_lon_east': 0.0,
 'bbox_lat_min': 0.0,
 'bbox_lat_max': 0.0,
 'publication_date': '',
 'doi': '',
 'crs': '',
 'shortname': 'Some name',
 'map_type': ['Integrated',
  'Morphologic',
  'Compositional',
  'Digital model',
  'Stratigraphic',
  'Geo-structural'],
 'target': 'Mars',
 'gmap_id': 'GMAP_IMCDSG_Mars_Some-name'}

In [8]:
# with output_panel:
#         print(form.to_json())
        
